<a href="https://colab.research.google.com/github/ton29ton29ton/humanism.site/blob/main/Movie_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# --- 必要ライブラリとGoogle Driveマウント ---
!pip install yt-dlp ffmpeg-python -q
from google.colab import drive
import os, shutil, re
import yt_dlp

# Google Drive マウント（既にマウント済ならスキップ）
try:
    drive.mount('/content/drive')
except:
    print("✅ Google Drive はすでにマウントされています。")

# --- ユーザー入力受付 ---
url = input("🔗 動画またはプレイリストのURLを入力してください:\n").strip()

mode = input("🎵 MP3 or 🎥 MP4 を選んでください（mp3 / mp4）: ").strip().lower()
if mode not in ["mp3", "mp4"]:
    print("❌ 無効な形式です。mp3 または mp4 を選んでください。")
    raise SystemExit

if mode == "mp3":
    audio_quality = input("🎧 音質を選んでください（最高 / 中 / 最低）: ").strip()
    quality_map = {"最高": "0", "中": "5", "最低": "9"}
    audio_quality_code = quality_map.get(audio_quality, "0")
else:
    video_quality = input("📺 動画画質を指定してください（例：best, 1080, 720, 480, 360 など）: ").strip()
    if video_quality.isdigit():
        format_code = f"bestvideo[height<={video_quality}]+bestaudio/best"
    else:
        format_code = "best"

folder_name = input("📁 Google Driveの保存フォルダ名を入力してください: ").strip() or "DL_Files"
keywords_raw = input("🧹 ファイル名から削除したい語句（カンマ区切り）: ").strip()
remove_keywords = [kw.strip() for kw in keywords_raw.split(",") if kw.strip()]

# --- 作業ディレクトリの準備 ---
download_dir = "/content/downloads"
if os.path.exists(download_dir):
    shutil.rmtree(download_dir)
os.makedirs(download_dir, exist_ok=True)

# --- yt-dlp オプションの構築 ---
outtmpl = f'{download_dir}/%(title)s.%(ext)s'
postprocessors = []

if mode == "mp3":
    postprocessors = [{
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'mp3',
        'preferredquality': audio_quality_code,
    }]
    format_code = "bestaudio"

ydl_opts = {
    'format': format_code,
    'outtmpl': outtmpl,
    'postprocessors': postprocessors,
    'quiet': False,
    'ignoreerrors': True,
    'noplaylist': False,
}

# --- ダウンロード処理 ---
with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    ydl.download([url])

# --- ファイル名整形（不要語句・空白・番号などを除去） ---
for file in os.listdir(download_dir):
    if mode == "mp3" and not file.lower().endswith(".mp3"):
        continue
    if mode == "mp4" and not file.lower().endswith(".mp4"):
        continue

    old_path = os.path.join(download_dir, file)
    name = file

    name = re.sub(r'^\d+\s*-\s*', '', name)  # 先頭番号削除
    for kw in remove_keywords:
        name = name.replace(kw, '')
    name = re.sub(r'[\s\u3000]+', '', name)  # 空白削除

    new_path = os.path.join(download_dir, name.strip())
    os.rename(old_path, new_path)

# --- Google Driveへ保存 ---
drive_target_path = f"/content/drive/MyDrive/{folder_name}"
os.makedirs(drive_target_path, exist_ok=True)

for file in os.listdir(download_dir):
    shutil.copy(os.path.join(download_dir, file), os.path.join(drive_target_path, file))

print(f"\n✅ 完了！Google Drive に保存されました → /MyDrive/{folder_name}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
🔗 動画またはプレイリストのURLを入力してください:
https://youtu.be/RWTBCZAwOSI?si=wziEr0bkVKh5amI0
🎵 MP3 or 🎥 MP4 を選んでください（mp3 / mp4）: Mp3
🎧 音質を選んでください（最高 / 中 / 最低）: 最高
📁 Google Driveの保存フォルダ名を入力してください: Live Core
🧹 ファイル名から削除したい語句（カンマ区切り）: 
[youtube] Extracting URL: https://youtu.be/RWTBCZAwOSI?si=wziEr0bkVKh5amI0
[youtube] RWTBCZAwOSI: Downloading webpage
[youtube] RWTBCZAwOSI: Downloading tv client config
[youtube] RWTBCZAwOSI: Downloading tv player API JSON
[youtube] RWTBCZAwOSI: Downloading ios player API JSON
[youtube] RWTBCZAwOSI: Downloading player 94f771d8-main
[youtube] RWTBCZAwOSI: Downloading m3u8 information
[info] RWTBCZAwOSI: Downloading 1 format(s): 251
[download] Destination: /content/downloads/【公式】尾崎豊 「太陽の破片」 (LIVE CORE IN TOKYO DOME 1988・9・12)【7thシングル】 YUTAKA OZAKI／Taiyo No Hahen.webm
[download] 100% of    7.42MiB in 00:00:00 at 21.88MiB/s  
[ExtractAudio] Destinat